In [60]:
import pandas as pd
import numpy as np
import pickle
import random
import os
import time
import matplotlib.pyplot as plt
from tqdm import tqdm
import torch
from torch.optim import lr_scheduler
import json

In [66]:
normalize_column = None
normalize_max = None
normalize_min = None
data_time_range = None
input_shape = None

def process_config(path):
    global model, normalize_column, normalize_max, normalize_min, data_time_range, input_shape
    
    print("load model and get preprocessing data")
    
    with open(path) as file:
        json_file = json.loads(file.read())[0]
    
    normalize_column = json_file["normalize_column"]
    normalize_max = np.array(json_file["normalize_max"])
    normalize_min = np.array(json_file["normalize_min"])
    data_time_range = json_file["data_time_range"] * 3600
    input_shape = json_file["input_shape"]
    
    print('finish loading')


def get_time_base_test_data(df_list):
    x = []
    zeros = np.zeros((input_shape[0], input_shape[1]))
    normalize_dist = normalize_max - normalize_min
    
    for d in df_list:
        # transform dict to dataframe
        d.reset_index(drop=True, inplace=True)
        
        start_time = d['start_event_time'][0]
        for end_index, last_event_time in enumerate(d['last_event_time']):
            if (end_index >= (input_shape[0] - 1)) or ((last_event_time - start_time) > data_time_range):
                break
        
        d = d[normalize_column].iloc[: end_index + 1].values
        d = (d - normalize_min) / normalize_dist
        
        temp = zeros.copy()
        temp[:d.shape[0]] = d
    
        x.append(temp)
    
    return np.array(x)


def build_loader(path, target):
    data = pickle.load(open(path, 'rb'))
    data = get_time_base_test_data(data)
    data = torch.FloatTensor(data)
    data = data.view(data.shape[0], 1, data.shape[1], data.shape[2])
    
    data_loader = model.DataLoader(data, torch.LongTensor([target] * len(data)))
    loader = Data.DataLoader(
        dataset=data_loader,
        batch_size=1,
        shuffle=False,
        num_workers=0
    )
    return loader

def test(data, target, gcn):
    
    correct_num = 0
    total_num = float(len(data))

    data_loader = model.DataLoader(data, torch.LongTensor([target] * len(data)))

    loader = Data.DataLoader(
        dataset=data_loader,
        batch_size=MINIBATCH_SIZE,
        shuffle=True,
        pin_memory = True,
        num_workers=0
    )
    
    with torch.no_grad():
        for inputs, labels in loader:
            # get data
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = gcn(inputs)
            _, preds = torch.max(outputs, 1)
            loss = criterion(outputs, labels)
            correct_num += torch.sum(preds == labels.data)
            
    print('total_num:', total_num)
    print('correct_num:', correct_num.item())
    print('acc:', correct_num.item() / total_num)
        

def test_new(loader, test_model):
    
    total_num = len(loader) 
    correct_num = 0
    with torch.no_grad():
        for inputs, labels in loader:
            # get data
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = gcn(inputs)
            _, preds = torch.max(outputs, 1)
            loss = criterion(outputs, labels)
            correct_num += torch.sum(preds == labels.data)
            
    print('total_num:', total_num)
    print('correct_num:', correct_num.item())
    print('acc:', correct_num.item() / total_num)


process_config('../test_gcn/config.json')

load model and get preprocessing data
finish loading


In [41]:
# load train data
train_x = pickle.load(open('./train_data/train_x.pickle', 'rb'))
train_y = pickle.load(open('./train_data/train_y.pickle', 'rb'))
val_x = pickle.load(open('./train_data/val_x.pickle', 'rb'))
val_y = pickle.load(open('./train_data/val_y.pickle', 'rb'))

# load test data
normal_test = pickle.load(open('./test_data/normal_test.pickle', 'rb'))
abnormal_test = pickle.load(open('./test_data/abnormal_test.pickle', 'rb'))
apk_test = pickle.load(open('./test_data/apk_test.pickle', 'rb'))

In [52]:
import importlib
import model
import  torch.utils.data as Data
importlib.reload(model)

MINIBATCH_SIZE = 32
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


gcn = model.MyGCN(h=20, w=7, output_shape=2)
gcn = gcn.to(device)
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(gcn.parameters(), lr=0.001, momentum=0.9)
exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.7)

dataloader = {'train': Data.DataLoader(dataset=model.DataLoader(train_x, train_y),
                                       batch_size=MINIBATCH_SIZE,
                                       shuffle=True,
                                       pin_memory=True,
                                       num_workers=6
                                      ),
              'val': Data.DataLoader(dataset=model.DataLoader(val_x, val_y),
                                     batch_size=MINIBATCH_SIZE,
                                     shuffle=True,
                                     pin_memory=True,
                                     num_workers=2
                                    ),
             }

In [53]:
from torch.autograd import Variable
debug = False
since = time.time()
EPOCH = 30

train_len = float(len(train_x))
val_len = float(len(val_x))

for epoch in range(EPOCH): 
    print('Epoch [', epoch,' / ',EPOCH,'] : ')
    
    for mode in ['train', 'val']:
        if(mode == 'train'):
            gcn.train()
        else:
            gcn.eval()

        running_loss = 0.0
        running_corrects = 0
        
        for inputs, labels in tqdm(dataloader[mode]):
            # get data
            inputs = inputs.to(device)
            labels = labels.to(device)
            optimizer.zero_grad()
            
            with torch.set_grad_enabled(mode == 'train'):
                outputs = gcn(inputs)
                _, preds = torch.max(outputs, 1)
                loss = criterion(outputs, labels)

                # backward + optimize only if in training phase
                if mode == 'train':
                    loss.backward()
                    optimizer.step()
                    
            running_loss += loss.item()
            running_corrects += torch.sum(preds == labels.data)
            if debug: break
                
        if mode == 'train':
            exp_lr_scheduler.step()
            print('Training Loss:', running_loss / train_len)
            print('Training Acc:', running_corrects.item() / train_len)
        else:
            print('Validation Loss:', running_loss / val_len)
            print('Validation Acc:', running_corrects.item() / val_len)    

    time_elapsed = time.time() - since
    print('Epoch %d finish in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))

        

print('結束訓練')
time_elapsed = time.time() - since
print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))

  0%|                                                                                         | 0/3272 [00:00<?, ?it/s]

Epoch [ 0  /  30 ] : 


  0%|                                                                                         | 0/1403 [00:00<?, ?it/s]

Training Loss: 0.017295454047542878
Training Acc: 0.7495511155256723


  0%|                                                                                         | 0/3272 [00:00<?, ?it/s]

...

Validation Loss: 0.011779418118939275
Validation Acc: 0.9350849043989838
Epoch %d finish in 19m 26s
Epoch [ 29  /  30 ] : 


  0%|                                                                                         | 0/1403 [00:00<?, ?it/s]

Training Loss: 0.012148069112201865
Training Acc: 0.9232025519559902


100%|█████████████████████████████████████████████████████████████████████████████| 1403/1403 [00:06<00:00, 219.58it/s]

Validation Loss: 0.011765871656399527
Validation Acc: 0.9367116815973615
Epoch %d finish in 20m 6s
結束訓練
Training complete in 20m 6s


In [55]:
test(normal_test, 0, gcn)

total_num: 40000.0
correct_num: 35917
acc: 0.897925


In [56]:
test(abnormal_test, 1, gcn)

total_num: 40000.0
correct_num: 37567
acc: 0.939175


In [57]:
test(apk_test, 1, gcn)

total_num: 40000.0
correct_num: 36826
acc: 0.92065


In [49]:
torch.save(gcn.state_dict(), './my_gcn8.pt')

In [62]:
normal_loader = build_loader('../test_data/normal.pickle', 0)
abnormal_loader = build_loader('../test_data/abnormal.pickle', 1)
accelerate_loader = build_loader('../test_data/accelerate.pickle', 1)

In [67]:
test_new(normal_loader, gcn)

total_num: 329
correct_num: 316
acc: 0.9604863221884499


In [68]:
test_new(abnormal_loader, gcn)

total_num: 499
correct_num: 453
acc: 0.9078156312625251


In [69]:
test_new(accelerate_loader, gcn)

total_num: 6
correct_num: 3
acc: 0.5
